In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import dtypes
from solarimage.classifier.nn.trainer import DataSet, TrainIter
from solarimage.classifier.nn import tester

In [3]:
mnist = input_data.read_data_sets("example/MNIST_data/", one_hot=True)

Extracting example/MNIST_data/train-images-idx3-ubyte.gz


Extracting example/MNIST_data/train-labels-idx1-ubyte.gz
Extracting example/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting example/MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
trainer = TrainIter(100, 1000, 
                    images=mnist.train.images, 
                    labels=mnist.train.labels, 
                    learning_rate=1e-3,
                    reshape=False,
                    dtype=dtypes.uint8)
test_dataset = DataSet(mnist.test.images, mnist.test.labels,reshape=False,dtype=dtypes.uint8)

In [7]:
loss_list = trainer.run_simple_nn()

In [14]:
tester.runner(test_dataset, weights, bias)

0.91710001

In [8]:
loss_list

[2.3025854,
 2.3014612,
 2.3001819,
 2.2993898,
 2.2984869,
 2.2982974,
 2.2964525,
 2.2950163,
 2.2936251,
 2.2923844,
 2.2935104,
 2.291712,
 2.2900982,
 2.2901807,
 2.2876482,
 2.2879009,
 2.2858539,
 2.2853203,
 2.2829113,
 2.2828927,
 2.2821007,
 2.2820399,
 2.280411,
 2.27935,
 2.2793624,
 2.278594,
 2.2757778,
 2.2722428,
 2.273077,
 2.2723193,
 2.2707849,
 2.2697661,
 2.2673931,
 2.2687747,
 2.2674458,
 2.2651203,
 2.2663169,
 2.2631166,
 2.2584591,
 2.2573974,
 2.2557073,
 2.2562242,
 2.2551374,
 2.255821,
 2.2513447,
 2.2524564,
 2.252619,
 2.2490277,
 2.2566917,
 2.2530625,
 2.2516456,
 2.2453101,
 2.2493696,
 2.2522829,
 2.2407751,
 2.2436869,
 2.2495282,
 2.2421069,
 2.2358274,
 2.2431741,
 2.2383707,
 2.2473905,
 2.230638,
 2.2497222,
 2.2406659,
 2.2327721,
 2.2263138,
 2.2310946,
 2.2358122,
 2.2351878,
 2.2273972,
 2.2362852,
 2.2302494,
 2.2280154,
 2.2337754,
 2.2278063,
 2.2239292,
 2.2221124,
 2.2316427,
 2.212939,
 2.2221537,
 2.2209225,
 2.22721,
 2.2274418,
 2.2